In [2]:
import pandas as pd

In [3]:
TN_census_df = pd.read_csv('../data/TN_census_clean.csv')

In [4]:
TN_census_df = TN_census_df.drop(['index'], axis=1)
TN_census_df.head()


,name,Total:,"Less than $10,000","$10,000 to $14,999","$15,000 to $19,999","$20,000 to $24,999","$25,000 to $29,999","$30,000 to $34,999","$35,000 to $39,999","$40,000 to $44,999","$45,000 to $49,999","$50,000 to $59,999","$60,000 to $74,999","$75,000 to $99,999","$100,000 to $124,999","$125,000 to $149,999","$150,000 to $199,999","$200,000 or more"
0,United States,125736350,6192080,4743710,4235151,4587937,4701374,4608052,4479480,4465681,4518761,8700174,11528244,16085302,12442523,9024401,11075396,14348087
1,Tennessee,2713635,144639,122212,111419,115858,114639,115755,118563,112751,114744,211329,267360,357378,257232,169567,182209,197980
2,"Davidson County, TN",302663,16412,9418,8738,10533,10101,11093,12015,12406,12257,23897,30572,40128,28621,20349,23356,32767
3,Nashville-Davidson metropolitan government (ba...,291884,15884,9205,8513,10076,9884,10764,11709,12190,11940,23053,29456,39076,27794,19878,22496,29966
4,"Nashville-Davidson--Murfreesboro--Franklin, TN...",772128,31679,20742,21206,24055,24019,25253,28342,29017,29879,57630,75506,107195,84133,58273,70741,84458


In [5]:
income_ranges = {
    'Less than $10,000': (0, 9999),
    '$10,000 to $14,999': (10000, 14999),
    '$15,000 to $19,999': (15000, 19999),
    '$20,000 to $24,999': (20000, 24999),
    '$25,000 to $29,999': (25000, 29999),
    '$30,000 to $34,999': (30000, 34999),
    '$35,000 to $39,999': (35000, 39999),
    '$40,000 to $44,999': (40000, 44999),
    '$45,000 to $49,999': (45000, 49999),
    '$50,000 to $59,999': (50000, 59999),
    '$60,000 to $74,999': (60000, 74999),
    '$75,000 to $99,999': (75000, 99999),
    '$100,000 to $124,999': (100000, 124999),
    '$125,000 to $149,999': (125000, 149999),
    '$150,000 to $199,999': (150000, 199999),
    '$200,000 or more': (200000, 500000),
}

In [6]:


# Function to calculate the income range for each ZIP code
def calculate_income_range(df, income_ranges):
    range_data = []

    # Iterate through each row (each ZIP code)
    for idx, row in df.iterrows():
        zip_code = row['name']  # Using 'name' for the ZIP code column
        
        # Create a cumulative sum for counts across income ranges
        cumulative_sum = 0
        total_count = row['Total:']  # Assuming you have a 'Total' column
        
        # Find the income range where the 50th percentile (median) falls
        for income_range, (lower_bound, upper_bound) in income_ranges.items():
            count_in_range = row[income_range]
            cumulative_sum += count_in_range

            # Check if the cumulative sum has passed the halfway mark (median point)
            if cumulative_sum >= total_count / 2:
                # Append the income range instead of the specific median value
                range_data.append({'Zip Code': zip_code, 'Income Range': income_range})
                break


    return pd.DataFrame(range_data)

# Apply the function to TN_census_df
median_income_range_df = calculate_income_range(TN_census_df, income_ranges)

In [11]:
#Merge original data to grab total surveyed.
median_income_range_complete_data = pd.merge(TN_census_df, median_income_range_df, left_on='name', right_on='Zip Code',how='inner')

#Clean table and rename columns.
median_income_range_complete_data = median_income_range_complete_data[['Zip Code', 'Total:','Income Range']]
median_income_range_complete_data = median_income_range_complete_data.rename(columns={'Zip Code': 'location', 'Total:': 'survey_count', 'Income Range': 'median_income_range' })
values_to_remove = ['United States', 'Tennessee', 'Davidson County, TN', 
                    'Nashville-Davidson metropolitan government (balance), TN',
                    'Nashville-Davidson--Murfreesboro--Franklin, TN Metro Area']
median_income_range_complete_data = median_income_range_complete_data[~median_income_range_complete_data['location'].isin(values_to_remove)]
#Save to CSV.
median_income_range_complete_data.to_csv('census_median_income.csv', index=False)
